<a href="https://colab.research.google.com/github/WSH032/ChatGLM-webui/blob/main/Colab_ChatGLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #（一）克隆秋叶的库、安装依赖

!git clone https://github.com/Akegarasu/ChatGLM-webui
%cd /content/ChatGLM-webui
print(f"正在安装依赖，请耐心等待")
!pip install --upgrade -r requirements.txt  > /dev/null 2>&1
print(f"依赖安装完成")

# （二）选择一个方法并使用

## 2.1秋叶方法（WebUI）

In [ ]:
#@title ###2.1.1 设置参数并开始对话

extArgs=""

#@markdown 是否用CPU进行推理`CPU模式请选择chatglm-6b-int4-qe，不然会爆ram`
use_cpu = False #@param {type:"boolean"}
#启用CPU模型
if use_cpu:
  extArgs = extArgs + "--cpu "

#@markdown 选择模型`Colab免费用户只能使用int4和qe模型`，或者填入自定义模型路径`将会覆盖预设模型选择`
model_path = "THUDM/chatglm-6b-int4" #@param ["THUDM/chatglm-6b", "THUDM/chatglm-6b-int4", "THUDM/chatglm-6b-int4-qe"]
your_model_path = "" #@param {type:"string"}
#用自定义路径覆盖预设
if your_model_path:
  model_path = your_model_path

#@markdown 推理精度`留空则自动指定, fp32只有CPU可使用 ； int4、int8只有GPU能用`
precision = "" #@param ["", "fp32", "fp16", "int4", "int8"]
#指定精度
if precision:
  extArgs = extArgs + f"--precision={precision} "

#启动
!python webui.py --model-path={model_path} --listen --share {extArgs}

## 2.2WSH方法（Colab风格`暂不支持流式对话`）

In [ ]:
#@title ###2.2.1选择并启用模型

from transformers import AutoModel, AutoTokenizer
import gradio as gr

#@markdown 是否用CPU进行推理`CPU模式请选择chatglm-6b-int4-qe，不然会爆ram`
use_cpu = False #@param {type:"boolean"}

#@markdown 选择模型`Colab免费用户只能使用int4和qe模型`，或者填入自定义模型路径`将会覆盖预设模型选择`
model_path = "THUDM/chatglm-6b-int4" #@param ["THUDM/chatglm-6b", "THUDM/chatglm-6b-int4", "THUDM/chatglm-6b-int4-qe"]
your_model_path = "" #@param {type:"string"}
#用自定义路径覆盖预设
if your_model_path:
  model_path = your_model_path

if use_cpu:
  tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
  model = AutoModel.from_pretrained(model_path, trust_remote_code=True).float()
else:
  tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
  model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()

model = model.eval()

#初始化参数
history = []
count = 1
max_length = 2048
top_p = 0.7
temperature =  0.95
max_turns = 20

In [55]:
#@title ###2.2.2参数设置（对话中也可更改）
max_length = 2048 #@param {type:"number"}
top_p = 0.7 #@param {type:"slider", min:0, max:1, step:0.01}
temperature =  0.95 #@param {type:"slider", min:0, max:1, step:0.01}
max_turns = 20 #@param {type:"slider", min:1, max:256, step:1}

In [59]:
#@title ###2.2.3提问

if count > max_turns:
  history = []
  count = 1
  print(f"达到对话次数上限，历史对话记录已被清空")

print(f"第 {count} 次对话，到达 {max_turns} 次后，下一次对话前将删除历史对话记录")
ask = "" #@param {type:"string"}
response, history = model.chat(tokenizer, ask, history, max_length=max_length, top_p=top_p,temperature=temperature)
print(f"回答：{response}")
count = count +1



达到对话次数上限，历史对话记录已被清空
第 1 次对话，到达 1 次后，下一次对话前将删除历史对话记录
回答：去向同样地方的人很多


In [ ]:
#@title ###2.2.4 历史对话记录



#@markdown 显示对话历史
show_history = True #@param {type:"boolean"}
if show_history:
  for ask_contet,ans_content in history:
    print(f"用户： {ask_contet}")
    print(f"回答： {ans_content}")
    print(f"------------------------------------------------------")
#@markdown 手动清空对话历史
clear_history = False #@param {type:"boolean"}
if clear_history:
  history = []
  count = 1
  print(f"聊天记录已被清空")

## 2.3官方流式方法

In [ ]:
#@title ###2.3.1选择并启用模型

from transformers import AutoModel, AutoTokenizer
import gradio as gr

#@markdown 是否用CPU进行推理`CPU模式请选择chatglm-6b-int4-qe，不然会爆ram`
use_cpu = False #@param {type:"boolean"}

#@markdown 选择模型`Colab免费用户只能使用int4和qe模型`，或者填入自定义模型路径`将会覆盖预设模型选择`
model_path = "THUDM/chatglm-6b-int4" #@param ["THUDM/chatglm-6b", "THUDM/chatglm-6b-int4", "THUDM/chatglm-6b-int4-qe"]
your_model_path = "" #@param {type:"string"}
#用自定义路径覆盖预设
if your_model_path:
  model_path = your_model_path

if use_cpu:
  tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
  model = AutoModel.from_pretrained(model_path, trust_remote_code=True).float()
else:
  tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
  model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()

model = model.eval()


In [ ]:
#@title ###2.3.2开启对话

#@markdown 最大对话轮数
MAX_TURNS = 20 #@param {type:"slider", min:1, max:256, step:1}
MAX_BOXES = MAX_TURNS * 2



def predict(input, max_length, top_p, temperature, history=None):
    if history is None:
        history = []
    for response, history in model.stream_chat(tokenizer, input, history, max_length=max_length, top_p=top_p,
                                               temperature=temperature):
        updates = []
        for query, response in history:
            updates.append(gr.update(visible=True, value="用户：" + query))
            updates.append(gr.update(visible=True, value="ChatGLM-6B：" + response))
        if len(updates) < MAX_BOXES:
            updates = updates + [gr.Textbox.update(visible=False)] * (MAX_BOXES - len(updates))
        yield [history] + updates


with gr.Blocks() as demo:
    state = gr.State([])
    text_boxes = []
    for i in range(MAX_BOXES):
        if i % 2 == 0:
            text_boxes.append(gr.Markdown(visible=False, label="提问："))
        else:
            text_boxes.append(gr.Markdown(visible=False, label="回复："))

    with gr.Row():
        with gr.Column(scale=4):
            txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter", lines=11).style(
                container=False)
        with gr.Column(scale=1):
            max_length = gr.Slider(0, 4096, value=2048, step=1.0, label="Maximum length", interactive=True)
            top_p = gr.Slider(0, 1, value=0.7, step=0.01, label="Top P", interactive=True)
            temperature = gr.Slider(0, 1, value=0.95, step=0.01, label="Temperature", interactive=True)
            button = gr.Button("Generate")
    button.click(predict, [txt, max_length, top_p, temperature, state], [state] + text_boxes)
demo.queue().launch()